## Dependencies

---

In [1]:
import yaml
import re
from datetime import datetime
import pandas as pd

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException


In [3]:
# edge:object = webdriver.Edge(executable_path="./utils/msedgedriver.exe")
edge = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))
edge.maximize_window()

[WDM] - Downloading: 100%|██████████| 7.74M/7.74M [00:01<00:00, 4.96MB/s]


## Extract

---

### Functions:

In [4]:
def resources(name: str) -> dict[str:str]:
    """Read xpath.yml file that contains xpath's to scrape"""

    with open("xpath.yml", "r") as f:
        content: dict[str:str] = yaml.safe_load(f)
        return content[name]

In [5]:
def close_prompt(driver: object) -> None:
    """Close ad window"""

    button: str = "//a[@class='btn btn-subscription-light']"
    driver.find_element(by=By.XPATH, value=button).click()

In [6]:
def choose_category(category: str, driver: object) -> None:
    """Choose ad category"""

    driver.find_element(by=By.XPATH, value=category).click()

In [7]:
def click_element(element: object, path: str) -> None:
    """Click element on the current document"""

    button = element.find_element(by=By.XPATH, value=path)
    button.click()

In [8]:
def extract_feature_in(element: object, path: str) -> str:
    """Extract ad feature where current browser page is in"""

    feature: str = element.find_element(by=By.XPATH, value=path)
    return feature

In [9]:
def extract_feature_out(element: object, path: str) -> list[str]:
    """Extract ad feature from list of ads (not within the ad)"""

    feature = []

    try:
        feature: str = element.find_elements(by=By.XPATH, value=path)
        feature = [data_point.text for data_point in feature]
        return feature
    except Exception:
        feature.append("Web wasn't able to scrape the feature")

In [10]:
def persistence(dataframe: pd.DataFrame) -> None:
    """Generate .csv file from data scraped"""

    file_name = datetime.today().strftime("%B-%d")
    dataframe.to_csv(f"persistence/{file_name}.csv", index=False)

In [11]:
def get_len(features: list[list[str]]) -> bool:
    """
    Check if all features has the same length to create new df
    :param features: list of features extracted with scraping
    :return: a list with all lengths where first place is the len of the first element and the mostly accepted.
    """

    base_len = len(features[0])
    lengths = []
    lengths.append(base_len)
    for feature in features:
        if len(feature) == base_len:
            continue
        else:
            lengths.append(len(feature))
    return lengths

In [12]:
def check_len(lengths: list[int]) -> bool:
    """
    Check if all elements in list has the same value.
    :param lengths: list of lengths
    :return: a boolean
    """
    for length in lengths[1:]:
        if length != lengths[0]:
            return False
        return True

Variables pool.

In [13]:
descriptions: list[str] = []
prices: list[int] = []
models: list[str] = []
years: list[int] = []
kilometers: list[int] = []
engines: list[str] = []

# seconds
DELAY: int = 1
SEDAN = resources("cars_sedan")

In [14]:
SEDAN["year"]

'/html/body/section/div[5]/div/div/div/ul/li[1]/span[3]'

Get web & close initial prompt (if is required).

In [15]:
edge.get(SEDAN["url"])
# close_prompt(edge)

Extract car features.

In [16]:
elements = edge.find_elements(by=By.XPATH, value=SEDAN["ad"])
elements[0].click()

try:
    for element in range(len(elements)):
        dynamic_delay = WebDriverWait(edge, DELAY).until(
            EC.presence_of_element_located((By.XPATH, SEDAN["description"])))

        brand_str = extract_feature_in(edge, SEDAN["brand"]).text
        model_str = extract_feature_in(edge, SEDAN["model"]).text
        models.append(brand_str + " " + model_str)

        descriptions.append(extract_feature_in(edge, SEDAN["description"]).text)
        prices.append(extract_feature_in(edge, SEDAN["price"]).text)
        kilometers.append(extract_feature_in(edge, SEDAN["kilometers"]).text)
        engines.append(extract_feature_in(edge, SEDAN["engine"]).text)
        years.append(extract_feature_in(edge, SEDAN["year"]).text)

        click_element(edge, SEDAN["next"])
except TimeoutException as TE:
    descriptions.append("Web wasn't able to scrape descriptions")
except NoSuchElementException as NSE:
    click_element(edge, SEDAN["exit"])

## Export

---

### Functions:

In [17]:
def parse_price(dataframe: pd.DataFrame, feature: str, pattern: str) -> None:
    """
    Parse financial features like prices. Works with $#,### (not dot)
    :param dataframe: object towards apply transformation
    :param feature: feature name towards apply transformation
    :param pattern: feature pattern to transform
    """
    dataframe[feature] = dataframe[feature] \
        .str.extract(pattern) \
        .applymap(lambda price: int(price.replace(",", "")))


In [18]:
def remove_chars(dataframe: pd.DataFrame, feature: str, wildcards: str) -> pd.Series:
    """
    Remove escaped characters from features like indented large extracts of text
    :param dataframe: object towards apply transformation
    :param feature: feature name towards apply transformation
    :param wildcards: escaped characters to remove
    :return: Serie without escaped characters
    """
    return (dataframe
    .apply(
        lambda row: row[feature]
        , axis=1
    )
    .apply(
        lambda extracted_f: re.sub(wildcards, r' ', extracted_f)
    )
    )


In [19]:
PRICE_RG: str = r"^\$(?P<prices>\d+,?\d+)?"
KM_REG: str = r"(?P<km>^\d+)"
CHARACTER_RG: str = '(\n|\r)+'

Check that all features has the same length.

In [20]:
lengths = [len(engines), len(models), len(years), len(kilometers), len(prices), len(descriptions)]
lengths

[31, 31, 31, 31, 31, 31]

Create dataframe to transform data extracted

In [21]:
if check_len(lengths):
    df_temp = pd.DataFrame({
        "model": models,
        "year": years,
        "kilometers": kilometers,
        "engine": engines,
        "price": prices,
        "description": descriptions
    })

    # Parse features. String to number and raw string with characters to structured string.
    parse_price(df_temp, "price", PRICE_RG)
    df_temp["kilometers"] = df_temp["kilometers"].str.extract(KM_REG).applymap(int)
    df_temp["year"] = df_temp["year"].map(int)
    df_temp["description"] = remove_chars(df_temp, "description", CHARACTER_RG)

    persistence(df_temp)


In [22]:
df_temp

,model,year,kilometers,engine,price,description
0,Kia Rio,2017,142000,Gasolina,7500,Precio negociable: si DESCRIPCIÓN CALIFICACION...
1,Mazda 3,2015,108,Gasolina,8400,"Se vende calidad de vehículo, excelente rendim..."
2,Toyota Corolla,2001,93000,Gasolina,3600,DESCRIPCIÓN CALIFICACIONES SEGURIDAD Vendo Cor...
3,Jeep Renegade,2019,31000,Gasolina,11500,Precio negociable: si Color : Naranjado
4,BMW 328,2015,110000,Gasolina,16000,"Color : Beige Se agarra carro más ribete, nada..."
5,Dodge Dart,2015,74000,Gasolina,7500,Garantía: Como es visto no hay garantía Financ...
6,Ford Focus,2007,200000,Gasolina,3000,Garantía: Como es visto no hay garantía Color ...
7,otros + DATSUN,1980,1000000,Gasolina,2000,Color : AZUL NEGRO DATSUN MODELO 210 CLASICO A...
8,Chevrolet Aveo,2011,133950,Gasolina,4900,Precio negociable: si DESCRIPCIÓN ARTÍCULOS CA...
9,Mitsubishi Lancer,2013,158000,Gasolina,6700,Precio negociable: si Color : Blanco Mitsubish...
